In [6]:
!pip install numpy opencv-python open3d scipy trimesh vedo



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   --- -----------------------

In [1]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

# Load fundus and mask images
fundus_img = cv2.imread("ORIGA/Images/023.jpg")  # Load the fundus image
mask_img = cv2.imread("img_1.png", cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale

# Resize mask to match fundus image dimensions
mask_img = cv2.resize(mask_img, (fundus_img.shape[1], fundus_img.shape[0]))

# Apply thresholding to create a binary mask
_, binary_mask = cv2.threshold(mask_img, 50, 255, cv2.THRESH_BINARY)

# Normalize mask for height mapping
height_map = binary_mask / 255.0 * 50  # Scale the depth (adjust factor if needed)

# Get image dimensions
h, w = height_map.shape

# Generate 3D Point Cloud
points = []
colors = []

for i in range(h):
    for j in range(w):
        if binary_mask[i, j] > 0:  # Consider only affected regions
            z = height_map[i, j]  # Use mask intensity for depth
            points.append([j, i, z])  # Store point in 3D space

            # Assign color from fundus image
            b, g, r = fundus_img[i, j]
            colors.append([r / 255.0, g / 255.0, b / 255.0])

# Convert to Open3D format
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(points))
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))

# Save the point cloud (optional)
o3d.io.write_point_cloud("glaucoma_point_cloud.ply", pcd)

# Create a mesh using Poisson surface reconstruction
pcd.estimate_normals()
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# Smooth the mesh
mesh = mesh.filter_smooth_taubin(number_of_iterations=10)

# Visualize the 3D glaucoma region
o3d.visualization.draw_geometries([mesh], window_name="Glaucoma 3D Reconstruction")

# Save the mesh model
o3d.io.write_triangle_mesh("glaucoma_mesh.obj", mesh)

print("3D model saved as 'glaucoma_mesh.obj'. You can view it in 3D modeling software like Blender.")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
3D model saved as 'glaucoma_mesh.obj'. You can view it in 3D modeling software like Blender.


In [2]:
import cv2
import numpy as np
import open3d as o3d
import trimesh
from skimage import measure
import matplotlib.pyplot as plt

# Load Fundus and Mask Images
fundus_img = cv2.imread("ORIGA/Images/023.jpg")  # Load the fundus image
mask_img = cv2.imread("img_1.png", cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale

# Resize mask to match fundus image
mask_img = cv2.resize(mask_img, (fundus_img.shape[1], fundus_img.shape[0]))

# Apply thresholding to extract the affected region
_, binary_mask = cv2.threshold(mask_img, 50, 255, cv2.THRESH_BINARY)

# Normalize mask for height mapping
height_map = binary_mask / 255.0 * 50  # Scale the depth (adjust for better height mapping)

# Create 3D Voxel Grid
h, w = height_map.shape
volume = np.zeros((h, w, 50))  # Creating an empty volume for marching cubes

for i in range(h):
    for j in range(w):
        if binary_mask[i, j] > 0:  # Consider only affected regions
            z = int(height_map[i, j])  # Height value from mask
            volume[i, j, :z] = 1  # Fill voxel grid

# Apply Marching Cubes for Mesh Generation
verts, faces, normals, values = measure.marching_cubes(volume, level=0)

# Convert to Open3D Mesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

# Smooth the mesh
mesh = mesh.filter_smooth_taubin(number_of_iterations=10)

# Estimate Normals for better visualization
mesh.compute_vertex_normals()

# Save and Visualize the Mesh
o3d.io.write_triangle_mesh("glaucoma_precise_mesh.obj", mesh)
o3d.visualization.draw_geometries([mesh], window_name="Precise Glaucoma 3D Reconstruction")

print("3D model saved as 'glaucoma_precise_mesh.obj'. You can open it in Blender or MeshLab.")


[Open3D WARNING] Write OBJ can not include triangle normals.
3D model saved as 'glaucoma_precise_mesh.obj'. You can open it in Blender or MeshLab.


In [8]:
import cv2
import numpy as np
import open3d as o3d
import trimesh
import vedo
from skimage import measure
import matplotlib.pyplot as plt

# Load Fundus and Mask Images
fundus_img = cv2.imread("ORIGA/Images/023.jpg")  # Fundus image
mask_img = cv2.imread("img_1.png", cv2.IMREAD_GRAYSCALE)  # Mask image in grayscale

# Resize the mask to match fundus image
mask_img = cv2.resize(mask_img, (fundus_img.shape[1], fundus_img.shape[0]))

# Apply contrast enhancement (helps detect edges better)
mask_img = cv2.equalizeHist(mask_img)

# Threshold the mask to extract affected region
_, binary_mask = cv2.threshold(mask_img, 50, 255, cv2.THRESH_BINARY)

# Normalize mask for height mapping (scaling depth)
height_map = binary_mask / 255.0 * 50  # Scale depth (adjust as needed)

# Create 3D Voxel Grid
h, w = height_map.shape
depth = 50  # Depth layers for 3D effect
volume = np.zeros((h, w, depth))  # 3D voxel space

for i in range(h):
    for j in range(w):
        if binary_mask[i, j] > 0:  # Affected region
            z = int(height_map[i, j])  # Depth from mask intensity
            volume[i, j, :z] = 1  # Fill voxel grid

# Apply Marching Cubes for Precise Surface Generation
verts, faces, normals, values = measure.marching_cubes(volume, level=0)

# Convert to Open3D Mesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

# Apply Smoothing (to remove jagged edges)
mesh = mesh.filter_smooth_taubin(number_of_iterations=15)

# Estimate Normals for better lighting effects
mesh.compute_vertex_normals()

# Convert Open3D mesh to Trimesh for further processing
tri_mesh = trimesh.Trimesh(vertices=np.asarray(mesh.vertices), faces=np.asarray(mesh.triangles))

# **Texture Mapping from Fundus Image**
texture_coords = []
colors = []

for v in mesh.vertices:
    x, y, z = int(v[0]), int(v[1]), int(v[2])  # Map to image coordinates
    if 0 <= x < w and 0 <= y < h:
        b, g, r = fundus_img[y, x]  # Get color from fundus image
        colors.append([r / 255.0, g / 255.0, b / 255.0])  # Normalize colors
    else:
        colors.append([1, 1, 1])  # Default white color

# Assign texture color to mesh
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# Save Mesh for Medical Analysis
o3d.io.write_triangle_mesh("glaucoma_medical_model.obj", mesh)

# **🔹 INTERACTIVE 3D VIEWER FOR DOCTORS**
vedo_mesh = vedo.Mesh([verts, faces]).lw(0.1).cmap("coolwarm", values)
vedo.show(vedo_mesh, axes=1, viewup="z", bg="black", title="Medical 3D Glaucoma Visualization")

print("✅ 3D medical model saved as 'glaucoma_medical_model.obj'. Open in Blender or 3D viewer for further analysis.")


[Open3D WARNING] Write OBJ can not include triangle normals.
✅ 3D medical model saved as 'glaucoma_medical_model.obj'. Open in Blender or 3D viewer for further analysis.


In [13]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage import measure

# Load fundus and mask images
fundus_img = cv2.imread("ORIGA/Images/023.jpg")  # Fundus image
mask_img = cv2.imread("img_1.png", cv2.IMREAD_GRAYSCALE)  # Mask image in grayscale

# Resize mask to match fundus image dimensions
mask_img = cv2.resize(mask_img, (fundus_img.shape[1], fundus_img.shape[0]))

# Normalize and enhance mask for smooth height mapping
mask_blurred = gaussian_filter(mask_img, sigma=3)  # Apply Gaussian smoothing
height_map = mask_blurred / 255.0 * 50  # Scale depth values (tune this value)

# Generate a 3D mesh grid
h, w = height_map.shape
x = np.linspace(0, w, w)
y = np.linspace(0, h, h)
X, Y = np.meshgrid(x, y)

# Convert height map into a point cloud
Z = height_map  # Depth values from mask
points = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=-1)

# Create a 3D Mesh using Marching Cubes
# Expand 2D height map into a 3D volume
depth_slices = 10  # Number of depth layers (tune for better results)
volume = np.repeat(height_map[:, :, np.newaxis], depth_slices, axis=2)  # Create a 3D volume

# Apply Marching Cubes on the 3D volume
verts, faces, normals, _ = measure.marching_cubes(volume, level=1)  # Level=1 ensures valid surface


# Convert to Open3D mesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

# Apply Smoothing for better results
mesh = mesh.filter_smooth_taubin(number_of_iterations=15)

# Generate colors from fundus image
colors = []
for v in verts:
    x, y = int(v[0]), int(v[1])
    if 0 <= x < w and 0 <= y < h:
        b, g, r = fundus_img[y, x]  # Get color from fundus image
        colors.append([r / 255.0, g / 255.0, b / 255.0])
    else:
        colors.append([1, 1, 1])  # Default white

# Apply vertex colors
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# Save and visualize
o3d.io.write_triangle_mesh("glaucoma_3d.obj", mesh)
o3d.visualization.draw_geometries([mesh], window_name="Glaucoma 3D Model")
